# Intro and rationale

Since breast density decreases with age, and is consistent over time, we want to create an algorithm that predicts the age of a patient from digital mammograms, to then compare the predicted age with the actual age.

# Data preparation: 
CC images from 500 patients with normal outcome from the OPTIMAM dataset. 

For a first experiment we categorise age in two groups: under 55 and over 55. We pay attention to select the same number for each group. 

We convert them to .png, divide them in two directories (under55 and over55), and then divide them in test, val and train subfolders.


# Algorithms 
DAAR topology 

https://arxiv.org/pdf/2201.02295.pdf

Persistent Homology for Breast Tumor Classification using Mammogram Scans